In [0]:
%pip install mlflow-skinny==1.28

Python interpreter will be restarted.
  Attempting uninstall: mlflow-skinny
    Found existing installation: mlflow-skinny 1.27.0
    Uninstalling mlflow-skinny-1.27.0:
      Successfully uninstalled mlflow-skinny-1.27.0
Python interpreter will be restarted.


In [0]:
from typing import List
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType, StructField, IntegerType

data: List = [
              (1, 3, 1),
  (1, 3, 1),
  (1, 3, 1),
  (1, 3, 1),
  (1, 3, 1),
              (2, 4, 1),
              (2, 3, 1),
              (3, 3, 1),
              (3, 4, 1)
  ]

schema: StructType = StructType([ \
    StructField("utilisateur_identifiant", IntegerType(), True), \
    StructField("diamant_identifiant", IntegerType(), True),
    StructField("nombre_de_fois_achetes", IntegerType(), True)
  ])

diamants_pre_features: DataFrame = spark.createDataFrame(data=data,schema=schema)


from pyspark.ml.recommendation import ALS, ALSModel

als: ALS = ALS(
  userCol="utilisateur_identifiant", 
  itemCol="diamant_identifiant", 
  ratingCol="nombre_de_fois_achetes",
  implicitPrefs=True,
  alpha=40,
  nonnegative=True
)
model: ALSModel = als.fit(diamants_pre_features)

import mlflow
mlflow.set_experiment("/nastasia/ALS_experiment")

with mlflow.start_run() as last_run:
  mlflow.spark.log_model(model, "als_exp")

from mlflow.tracking import MlflowClient
# Get last run from Mlflow experiment
client = MlflowClient()

model_experiment_id = client.get_experiment_by_name("/nastasia/ALS_experiment").experiment_id

runs = client.search_runs(
        model_experiment_id, order_by=["start_time DESC"]
)

run_uuid = runs[0].info.run_uuid

# can be loaded from s3
# model = ALSModel.load(sources_jobs['ALS_model'])
loaded_model = mlflow.spark.load_model(f"runs:/{run_uuid}/als_exp")

2022/10/03 11:44:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmplav5lz8q, flavor: spark), fall back to return ['pyspark==3.3.0']. Set logging level to DEBUG to see the full traceback.
2022/10/03 11:44:39 INFO mlflow.spark: 'runs:/e66f017a224a45f5bd9af3510d85d5da/als_exp' resolved as 'dbfs:/databricks/mlflow-tracking/584661241112115/e66f017a224a45f5bd9af3510d85d5da/artifacts/als_exp'


---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-584661241112111> in <cell line: 58>()
     56 # can be loaded from s3
     57 # model = ALSModel.load(sources_jobs['ALS_model'])
---> 58 loaded_model = mlflow.spark.load_model(f"runs:/{run_uuid}/als_exp")

/local_disk0/.ephemeral_nfs/envs/pythonEnv-3c21b356-9cd5-407f-99c9-99eaac466b0b/lib/python3.9/site-packages/mlflow/spark.py in load_model(model_uri, dfs_tmpdir)
    784             get_databricks_profile_uri_from_artifact_uri(root_uri)
    785         ):
--> 786             return PipelineModel.load(mlflowdbfs_path)
    787 
    788     return _load_model(

/databricks/spark/python/pyspark/ml/util.py in load(cls, path)
    444     def load(cls, path: str) -> RL:
    445         """Reads an ML instance from the input path, a shortcut of `read().load(path)`."""
--> 446         return cls.read().load(path)
    447 
    448 

/da

In [0]:
%sh pip freeze

absl-py==1.0.0
argon2-cffi==20.1.0
astor==0.8.1
astunparse==1.6.3
async-generator==1.10
attrs==21.2.0
azure-core==1.22.1
azure-cosmos==4.2.0
backcall==0.2.0
backports.entry-points-selectable==1.1.1
bcrypt==4.0.0
black==22.3.0
bleach==4.0.0
blis==0.7.8
boto3==1.21.18
botocore==1.24.18
cachetools==5.2.0
catalogue==2.0.8
certifi==2021.10.8
cffi==1.14.6
chardet==4.0.0
charset-normalizer==2.0.4
click==8.0.3
cloudpickle==2.0.0
cmdstanpy==0.9.68
configparser==5.2.0
convertdate==2.4.0
cryptography==3.4.8
cycler==0.10.0
cymem==2.0.6
Cython==0.29.24
databricks-automl-runtime==0.2.10
databricks-cli==0.17.0
dbl-tempo==0.1.12
dbus-python==1.2.16
debugpy==1.4.1
decorator==5.1.0
defusedxml==0.7.1
dill==0.3.4
diskcache==5.4.0
distlib==0.3.5
distro==1.4.0
distro-info===0.23ubuntu1
entrypoints==0.3
ephem==4.1.3
facets-overview==1.0.0
fasttext==0.9.2
filelock==3.3.1
Flask==1.1.2
flatbuffers==1.12
fsspec==2021.8.1
future==0.18.2
gast==0.4.0
gitdb==4.0.9
GitPython==3.1.27
google-auth==2.6.0
google-auth-oau